# User DEM Sampling

This notebook shows how to sample a user-provided DEM and compare it with SlideRule's supported ESA Copernicus dataset path.

ESA Copernicus supports AWS S3 protocol but is hosted at the San Diego Supercomputer Center (SDSC) using S3-compatible object storage. This means it can be accessed using either S3 protocol paths or HTTPS URLs.

In SlideRule, the supported dataset asset (`esa-copernicus-30meter`) is implemented via the GDAL `/vsis3/` driver. The same VRT can also be accessed over HTTPS using GDAL `/vsicurl/`, which makes it a good candidate to compare supported-dataset sampling with user-specified URL DEM sampling.

In [ ]:
import sliderule
from sliderule import raster

sliderule.init(url="slideruleearth.io", verbose=False)
# sliderule.init(url="localhost", organization=None, verbose=False)

In [ ]:
def sample_one(asset, parms=None):
    parms = parms or {}
    gdf = raster.sample(asset, [[-108.1, 39.1]], parms=parms)
    if len(gdf) == 0:
        raise RuntimeError(f"No samples returned for asset={asset}, parms={parms}")
    return float(gdf["value"].iat[0]), gdf["file"].iat[0]

The COP30 VRT used here advertises a 2D CRS that does not fully represent the required vertical datum handling. In SlideRule's built-in `esa-copernicus-30meter` path, CRS handling is overridden to `EPSG:9055+3855`, and pixel elevations are vertically adjusted using the appropriate PROJ grid.
Sampling in this path is performed through the GDAL `/vsis3/` driver.

In [ ]:
dataset_val, dataset_file = sample_one("esa-copernicus-30meter")
print(f"  value={dataset_val:.2f}")
print(f"  file={dataset_file}")

Now access the same COP30 VRT as a user-provided DEM URL (`user-url-raster`) using only `url`.
In this mode, SlideRule uses the raster's native CRS metadata. Because this VRT CRS is incomplete for vertical adjustment, the returned elevation is typically biased (about +15 m at this test point).

In [ ]:
url = "https://opentopography.s3.sdsc.edu/raster/COP30/COP30_hh.vrt"

user_raw_val, user_raw_file = sample_one("user-url-raster", {"url": url})
print(f"  value={user_raw_val:.2f}")
print(f"  file={user_raw_file}")
print(f"  delta_vs_dataset={user_raw_val - dataset_val:+.2f} m")

Continue using the user-specified DEM, but provide `target_crs` so SlideRule can apply the correct vertical reference handling.

In [ ]:
user_target_val, user_target_file = sample_one("user-url-raster", {"url": url, "target_crs": "EPSG:9055+3855"})
print(f"  value={user_target_val:.2f}")
print(f"  file={user_target_file}")
print(f"  delta_vs_dataset={user_target_val - dataset_val:+.2f} m")

Another way to achieve the same correction is to provide an explicit `proj_pipeline`.
In this mode, SlideRule keeps the raster CRS metadata but uses your pipeline as the coordinate operation instead of auto-building a transform from source/target CRS definitions.

In [ ]:
pipeline = "+proj=pipeline \
            +step +proj=axisswap +order=2,1 \
            +step +proj=unitconvert +xy_in=deg +xy_out=rad \
            +step +proj=cart +ellps=GRS80 \
            +step +inv +proj=helmert +x=-0.0007 +y=-0.0012 +z=0.0261 +rx=0 +ry=0 +rz=0 \
                    +s=-0.00212 +dx=-0.0001 +dy=-0.0001 +dz=0.0019 +drx=0 +dry=0 +drz=0 \
                    +ds=-0.00011 +t_epoch=2010 +convention=position_vector \
            +step +inv +proj=cart +ellps=WGS84 \
            +step +inv +proj=vgridshift +grids=us_nga_egm08_25.tif +multiplier=1 \
            +step +proj=unitconvert +xy_in=rad +xy_out=deg \
            +step +proj=axisswap +order=2,1"

user_pipeline_val, user_pipeline_file = sample_one("user-url-raster", {"url": url, "proj_pipeline": pipeline})
print(f"  value={user_pipeline_val:.2f}")
print(f"  file={user_pipeline_file}")
print(f"  delta_vs_dataset={user_pipeline_val - dataset_val:+.2f} m")

In [ ]:
print("Summary")
print(f"  dataset asset value      : {dataset_val:.2f}")
print(f"  user url only value      : {user_raw_val:.2f}")
print(f"  user + target_crs value  : {user_target_val:.2f}")
print(f"  user + pipeline value    : {user_pipeline_val:.2f}")

For ICESat-2-driven sampling workflows (ATLxx inputs), prefer setting `target_crs` when the VRT/raster requires a specific destination CRS, because the source CRS is already determined by the product stream.

Pipeline-only usage can be sufficient for simple value sampling, but remember that SlideRule enforces traditional GIS axis order (lon/lat), and raster CRS metadata is also used for metric operations such as resampling radius and slope/aspect calculations.

If `proj_pipeline` and `target_crs` are inconsistent, coordinate transforms may still execute while metric-derived outputs become physically incorrect. For robust scientific use, provide `target_crs`, or provide both a fully specified pipeline and a matching `target_crs`.